In [1]:
# all imports
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

# notebook settings
from IPython.display import display
pd.options.display.max_columns = None

## Read in the sales-valuation data and prepare for merge
* read in the sales-valuation pickle file
* derive SALE YEAR and SALE MONTH columns from SALE DATE column so that we can join on them

In [2]:
# make sure your dir structure looks like this:
# dsga1001/ (git repo) --> dsga1001/ --> ipython/ --> .ipynb files
# dsga1001-data/ --> .pkl files

# path for retrieving data files
# (adjust as needed if your local data directory setup is different)
path = '../../../dsga1001-data/'

In [3]:
# import the merged sales-valuation dataframe (the inner join version)
df = pd.read_pickle(path + 'inner-merged.pkl')
display(df)

,S_borough,S_neighborhood,S_building_class_category,S_block,S_lot,S_address,S_apt_number,S_zipcode,S_residential_units,S_commercial_units,S_total_units,S_land_sqft,S_gross_sqft,S_year_built,S_tax_class_at_sale,S_building_class_at_sale,S_sale_price,S_sale_date,bbl,bbl_block,bbl_lot,year,v_bble,v_boro,v_block,v_lot,v_owner,v_bldgcl,v_taxclass,v_ltfront,v_ltdepth,v_ext,v_stories,v_fullval,v_avland,v_avtot,v_staddr,v_postcode,v_bldfront,v_blddepth,v_avland2,v_avtot2,v_latitude,v_longitude,v_community_board,v_council_district,v_census_tract,v_bin,v_nta,v_new_georeferenced_column
0,2,BATHGATE,01 ONE FAMILY DWELLINGS,03028,0025,412 EAST 179TH STREET,,10457.0,1.0,0.0,1.0,1842.0,2048.0,1901.0,1,A5,355000,2013-07-08,2030280025,03028,0025,2013/14,2030280025,2,03028,0025,SANDRA FERNANDEZ,A5,1,18,100,E,3.0,258000,7496,8557,412 EAST 179 STREET,10457.0,18,32,NaN,NaN,40.850339,-73.898762,206.0,15.0,379.0,2011104.0,Mount Hope,POINT (-73.898762 40.850339)
1,2,BATHGATE,01 ONE FAMILY DWELLINGS,03039,0028,2329 WASHINGTON AVENUE,,10458.0,1.0,0.0,1.0,1103.0,1290.0,1910.0,1,A1,120000,2013-11-14,2030390028,03039,0028,2013/14,2030390028,2,03039,0028,"PERRY , EUGENE",A1,1,15,73,NaN,2.0,303000,5249,8460,2329 WASHINGTON AVENUE,10458.0,15,40,NaN,NaN,40.855982,-73.892503,206.0,15.0,385.0,2011364.0,Claremont-Bathgate,POINT (-73.892503 40.855982)
2,2,BATHGATE,01 ONE FAMILY DWELLINGS,03039,0028,2329 WASHINGTON AVENUE,,10458.0,1.0,0.0,1.0,1103.0,1290.0,1910.0,1,A1,474819,2013-05-20,2030390028,03039,0028,2012/13,2030390028,2,03039,0028,"PERRY , EUGENE",A1,1,15,73,NaN,2.0,334000,4762,8460,2329 WASHINGTON AVENUE,10458.0,15,40,NaN,NaN,40.855982,-73.892503,206.0,15.0,385.0,2011364.0,Claremont-Bathgate,POINT (-73.892503 40.855982)
3,2,BATHGATE,01 ONE FAMILY DWELLINGS,03046,0039,2075 BATHGATE AVENUE,,10457.0,1.0,0.0,1.0,1986.0,1344.0,1899.0,1,A1,210000,2013-03-12,2030460039,03046,0039,2012/13,2030460039,2,03046,0039,"VELEZ, PABLO",A1,1,21,94,NaN,2.0,206000,7137,7862,2075 BATHGATE AVENUE,10457.0,18,42,NaN,NaN,40.850459,-73.895151,206.0,15.0,395.0,2011486.0,Claremont-Bathgate,POINT (-73.895151 40.850459)
4,2,BATHGATE,01 ONE FAMILY DWELLINGS,03046,0052,2047 BATHGATE AVENUE,,10457.0,1.0,0.0,1.0,2329.0,1431.0,1901.0,1,A1,343116,2013-07-01,2030460052,03046,0052,2013/14,2030460052,2,03046,0052,MICHAEL T DONALD,A1,1,25,93,E,2.0,200000,6979,8308,2047 BATHGATE AVENUE,10457.0,20,30,NaN,NaN,40.850031,-73.895394,206.0,15.0,395.0,2011492.0,Claremont-Bathgate,POINT (-73.895394 40.850031)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340535,5,WOODROW,02 TWO FAMILY DWELLINGS,07316,0084,57 DARNELL LANE,None,10309.0,2.0,0.0,2.0,2311.0,1500.0,1995.0,1,B2,620000,2019-06-20,5073160084,07316,0084,2018/19,5073160084,5,07316,0084,SALVATORE ANNARUMMA,B2,1,26,88,NaN,2.0,518000,6690,27072,57 DARNELL LANE,10309.0,16,44,NaN,NaN,40.536003,-74.223213,503.0,51.0,226.0,5130720.0,Charleston-Richmond Valley-Tottenville,POINT (-74.223213 40.536003)
340536,5,WOODROW,02 TWO FAMILY DWELLINGS,07317,0119,73 ROBIN COURT,None,10309.0,2.0,0.0,2.0,4125.0,2160.0,1994.0,1,B2,625000,2019-05-21,5073170119,07317,0119,2018/19,5073170119,5,07317,0119,JOSEPH DIBENEDETTO,B2,1,25,165,NaN,3.0,527000,9683,29160,73 ROBIN COURT,10309.0,22,40,NaN,NaN,40.536140,-74.222012,503.0,51.0,226.0,5125758.0,Charleston-Richmond Valley-Tottenville,POINT (-74.222012 40.53614)
340537,5,WOODROW,02 TWO FAMILY DWELLINGS,07349,0004,51 PHEASANT LANE,None,10309.0,2.0,0.0,2.0,2123.0,1850.0,1998.0,1,B9,670000,2019-02-22,5073490004,07349,0004,2018/19,5073490004,5,07349,0004,"GUITIAN, JOSEPH",B9,1,23,91,NaN,2.0,581000,6988,30528,51 PHEASANT LANE,10309.0,14,52,NaN,NaN,40.532829,-74.222422,503.0,51.0,226.0,5135567.0,Charleston-Richmond Valley-Tottenville,POINT (-74.222422 40.532829)
340538,5,WOODROW,02 TWO FAMILY DWELLINGS,07349,0035,33 QUAIL LANE,None,10309.0,2.0,0.0,2.0,2255.0,2377.0,1998.0,1,B9,

In [4]:
# based on the sale date column (s_sale_date), create additional columns for s_sale_month and s_sale_year.
df['S_sale_month'] = pd.to_datetime(df['S_sale_date']).map(lambda saledate : saledate.month)
df['S_sale_year'] = pd.to_datetime(df['S_sale_date']).map(lambda saledate : saledate.year)

# verify
display(df[['S_sale_date', 'S_sale_month', 'S_sale_year']])

,S_sale_date,S_sale_month,S_sale_year
0,2013-07-08,7,2013
1,2013-11-14,11,2013
2,2013-05-20,5,2013
3,2013-03-12,3,2013
4,2013-07-01,7,2013
...,...,...,...
340535,2019-06-20,6,2019
340536,2019-05-21,5,2019
340537,2019-02-22,2,2019
340538,2019-03-08,3,2019


In [5]:
# check dtype of the new cols
print(df['S_sale_month'].dtype)
print(df['S_sale_year'].dtype)

int64
int64


## Read in the macroeconomic data and prepare for merge

In [6]:
# import the combined macroeconomic data from CSV file
macro = pd.read_csv(path + 'macroeconomic_data.csv')
display(macro)

,YEAR,MONTH,10Yr Treasury Constant Maturity Rate,30Yr Fixed Rate Mortgage Avg in US,Case-Shiller NY Home Price Index,Case-Shiller US National Home Price Index,Consumer Price Index,Core CPI,Effective Fed Funds Rate,Inflation Rate,Population Level,Real Gross Domestic Product,SP500 Index,Unemployment Rate,US Real Median Household Income,10Yr Treasury Constant Maturity Rate %Chg,30Yr Fixed Rate Mortgage Avg in US %Chg,Case-Shiller NY Home Price Index %Chg,Case-Shiller US National Home Price Index %Chg,Consumer Price Index %Chg,Core CPI %Chg,Effective Fed Funds Rate %Chg,Inflation Rate %Chg,Population Level %Chg,Real Gross Domestic Product %Chg,SP500 Index %Chg,Unemployment Rate %Chg,US Real Median Household Income %Chg
0,2011,1,3.391905,4.7550,167.189630,141.525,221.187,222.803,0.168387,3.156842,311023.0,15712.754000,1283.124286,9.1,57021.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011,2,3.577000,4.9525,166.614113,140.354,221.898,223.213,0.156786,3.066216,311173.0,15750.201333,1322.212500,9.0,57011.916667,0.054570,0.041535,-0.003442,-0.008274,0.003214,0.001840,-0.068897,-0.028708,0.000482,0.002383,0.030463,-0.010989,-0.000159
2,2011,3,3.414348,4.8360,165.520460,139.983,223.046,223.454,0.138710,2.975591,311333.0,15787.648667,1304.486087,9.0,57002.833333,-0.045472,-0.023523,-0.006564,-0.002643,0.005174,0.001080,-0.115291,-0.029556,0.000514,0.002378,-0.013407,0.000000,-0.000159
3,2011,4,3.453333,4.8400,167.123040,140.006,224.093,223.727,0.098000,2.884965,311502.0,15825.096000,1331.784762,9.1,56993.750000,0.011418,0.000827,0.009682,0.000164,0.004694,0.001222,-0.293488,-0.030456,0.000543,0.002372,0.020927,0.011111,-0.000159
4,2011,5,3.164091,4.6375,166.837997,139.898,224.806,224.175,0.093871,2.794340,311678.0,15823.630667,1337.982727,9.0,56984.666667,-0.083757,-0.041839,-0.001706,-0.000771,0.003182,0.002002,-0.042133,-0.031413,0.000565,-0.000093,0.004654,-0.010989,-0.000159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,2019,8,1.626364,3.6160,198.826921,209.813,256.294,264.180,2.125806,1.812210,328730.0,19179.149000,2897.498182,3.7,68703.000000,-0.208498,-0.039575,-0.009133,0.000892,0.000808,0.002250,-0.115317,0.000000,0.000514,0.001954,-0.032910,0.000000,0.000000
104,2019,9,1.690000,3.6050,201.053048,210.959,256.593,264.682,2.043000,1.812210,328897.0,19216.554000,2979.503810,3.5,68703.000000,0.039128,-0.003042,0.011196,0.005462,0.001167,0.001900,-0.038953,0.000000,0.000508,0.001950,0.028302,-0.054054,0.000000
105,2019,10,1.709130,3.6880,201.999229,211.800,257.229,264.987,1.829677,1.812210,329054.0,19253.959000,2977.675217,3.6,68703.000000,0.011320,0.023024,0.004706,0.003987,0.002479,0.001152,-0.104416,0.000000,0.000477,0.001946,-0.000614,0.028571,0.000000
106,2019,11,1.816190,3.6950,202.494800,212.687,257.824,265.501,1.553333,1.812210,329191.0,19253.959000,3107.224762,3.5,68703.000000,0.062640,0.001898,0.002453,0.004188,0.002313,0.001940,-0.151034,0.000000,0.000416,0.000000,0.043507,-0.027778,0.000000


In [7]:
# check the dtypes on the macro df year and month cols
print(macro['YEAR'].dtype)
print(macro['MONTH'].dtype)

int64
int64


## Merge the sales-valuation data with macroeconomic data
Perform an inner join (intersection) of sales-valuation and macroeconomic datasets on key: (SALE YEAR, SALE MONTH)

Reference:
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html
* https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#database-style-dataframe-or-named-series-joining-merging

In [8]:
# join the merged sales-valuation dataframe with the macroeconomic data on (year, month) of the SALE DATE
merged = pd.merge(df, macro, how='inner', left_on=['S_sale_year', 'S_sale_month'], right_on=['YEAR', 'MONTH'])

In [9]:
display(merged)

,S_borough,S_neighborhood,S_building_class_category,S_block,S_lot,S_address,S_apt_number,S_zipcode,S_residential_units,S_commercial_units,S_total_units,S_land_sqft,S_gross_sqft,S_year_built,S_tax_class_at_sale,S_building_class_at_sale,S_sale_price,S_sale_date,bbl,bbl_block,bbl_lot,year,v_bble,v_boro,v_block,v_lot,v_owner,v_bldgcl,v_taxclass,v_ltfront,v_ltdepth,v_ext,v_stories,v_fullval,v_avland,v_avtot,v_staddr,v_postcode,v_bldfront,v_blddepth,v_avland2,v_avtot2,v_latitude,v_longitude,v_community_board,v_council_district,v_census_tract,v_bin,v_nta,v_new_georeferenced_column,S_sale_month,S_sale_year,YEAR,MONTH,10Yr Treasury Constant Maturity Rate,30Yr Fixed Rate Mortgage Avg in US,Case-Shiller NY Home Price Index,Case-Shiller US National Home Price Index,Consumer Price Index,Core CPI,Effective Fed Funds Rate,Inflation Rate,Population Level,Real Gross Domestic Product,SP500 Index,Unemployment Rate,US Real Median Household Income,10Yr Treasury Constant Maturity Rate %Chg,30Yr Fixed Rate Mortgage Avg in US %Chg,Case-Shiller NY Home Price Index %Chg,Case-Shiller US National Home Price Index %Chg,Consumer Price Index %Chg,Core CPI %Chg,Effective Fed Funds Rate %Chg,Inflation Rate %Chg,Population Level %Chg,Real Gross Domestic Product %Chg,SP500 Index %Chg,Unemployment Rate %Chg,US Real Median Household Income %Chg
0,2,BATHGATE,01 ONE FAMILY DWELLINGS,03028,0025,412 EAST 179TH STREET,,10457.0,1.0,0.0,1.0,1842.0,2048.0,1901.0,1,A5,355000,2013-07-08,2030280025,03028,0025,2013/14,2030280025,2,03028,0025,SANDRA FERNANDEZ,A5,1,18,100,E,3.0,258000,7496,8557,412 EAST 179 STREET,10457.0,18,32,NaN,NaN,40.850339,-73.898762,206.0,15.0,379.0,2011104.0,Mount Hope,POINT (-73.898762 40.850339),7,2013,2013,7,2.579565,4.3700,167.898883,155.630,232.900,233.880,0.092581,1.543528,316405.0,16531.685000,1666.359565,7.3,58452.5,0.121550,0.073710,0.004885,0.009156,0.001957,0.002271,-0.011595,0.00857,0.000642,0.002598,0.029397,-0.026667,-0.001286
1,2,BATHGATE,01 ONE FAMILY DWELLINGS,03046,0052,2047 BATHGATE AVENUE,,10457.0,1.0,0.0,1.0,2329.0,1431.0,1901.0,1,A1,343116,2013-07-01,2030460052,03046,0052,2013/14,2030460052,2,03046,0052,MICHAEL T DONALD,A1,1,25,93,E,2.0,200000,6979,8308,2047 BATHGATE AVENUE,10457.0,20,30,NaN,NaN,40.850031,-73.895394,206.0,15.0,395.0,2011492.0,Claremont-Bathgate,POINT (-73.895394 40.850031),7,2013,2013,7,2.579565,4.3700,167.898883,155.630,232.900,233.880,0.092581,1.543528,316405.0,16531.685000,1666.359565,7.3,58452.5,0.121550,0.073710,0.004885,0.009156,0.001957,0.002271,-0.011595,0.00857,0.000642,0.002598,0.029397,-0.026667,-0.001286
2,2,BAYCHESTER,01 ONE FAMILY DWELLINGS,04724,0006,3487 KINGSLAND AVENUE,,10469.0,1.0,0.0,1.0,2288.0,1480.0,1925.0,1,A2,235000,2013-07-30,2047240006,04724,0006,2013/14,2047240006,2,04724,0006,LIVING TRUST OF DOLOR,A2,1,25,91,NaN,1.0,338000,12317,18753,3487 KINGSLAND AVENUE,10469.0,21,42,NaN,NaN,40.879025,-73.845478,212.0,12.0,386.0,2060252.0,Eastchester-Edenwald-Baychester,POINT (-73.845478 40.879025),7,2013,2013,7,2.579565,4.3700,167.898883,155.630,232.900,233.880,0.092581,1.543528,316405.0,16531.685000,1666.359565,7.3,58452.5,0.121550,0.073710,0.004885,0.009156,0.001957,0.002271,-0.011595,0.00857,0.000642,0.002598,0.029397,-0.026667,-0.001286
3,2,BAYCHESTER,01 ONE FAMILY DWELLINGS,04750,0048,3318 TIEMANN AVENUE,,10469.0,1.0,0.0,1.0,1710.0,1539.0,1950.0,1,A5,165000,2013-07-29,2047500048,04750,0048,2013/14,2047500048,2,04750,0048,"UMANAH, AHAZIAH",A5,1,18,95,G,2.0,283000,12060,16980,3318 TIEMANN AVENUE,10469.0,18,33,NaN,NaN,40.876008,-73.842809,212.0,12.0,358.0,2061379.0,Eastchester-Edenwald-Baychester,POINT (-73.842809 40.876008),7,2013,2013,7,2.579565,4.3700,167.898883,155.630,232.900,233.880,0.092581,1.543528,316405.0,16531.685000,1666.359565,7.3,58452.5,0.121550,0.073710,0.004885,0.009156,0.001957,0.002271,-0.011595,0.00857,0.000642,0.002598,0.029397,-0.026667,-0.001286
4,2,BAYCHESTER,01 ONE FAMILY DWELLINGS,04792,0012,2837 GUNTHER AVENUE,,10469.0,1.0,0.0,1.0,4750.0,1063.0,1910.0,1,A1,205000,2013-07

In [10]:
merged.shape

(340540, 80)

In [11]:
# sanity check the merge -- check consistency of date fields
display(merged[['S_sale_price', 'S_sale_date', 'S_sale_month', 'S_sale_year', 'YEAR', 'MONTH']])

,S_sale_price,S_sale_date,S_sale_month,S_sale_year,YEAR,MONTH
0,355000,2013-07-08,7,2013,2013,7
1,343116,2013-07-01,7,2013,2013,7
2,235000,2013-07-30,7,2013,2013,7
3,165000,2013-07-29,7,2013,2013,7
4,205000,2013-07-30,7,2013,2013,7
...,...,...,...,...,...,...
340535,635000,2019-06-11,6,2019,2019,6
340536,855000,2019-06-17,6,2019,2019,6
340537,870000,2019-06-28,6,2019,2019,6
340538,778961,2019-06-10,6,2019,2019,6


In [12]:
# drop the redundant YEAR and MONTH cols in the merged df
merged = merged.drop(['YEAR', 'MONTH'], 1)

In [13]:
# validate
display(merged)

,S_borough,S_neighborhood,S_building_class_category,S_block,S_lot,S_address,S_apt_number,S_zipcode,S_residential_units,S_commercial_units,S_total_units,S_land_sqft,S_gross_sqft,S_year_built,S_tax_class_at_sale,S_building_class_at_sale,S_sale_price,S_sale_date,bbl,bbl_block,bbl_lot,year,v_bble,v_boro,v_block,v_lot,v_owner,v_bldgcl,v_taxclass,v_ltfront,v_ltdepth,v_ext,v_stories,v_fullval,v_avland,v_avtot,v_staddr,v_postcode,v_bldfront,v_blddepth,v_avland2,v_avtot2,v_latitude,v_longitude,v_community_board,v_council_district,v_census_tract,v_bin,v_nta,v_new_georeferenced_column,S_sale_month,S_sale_year,10Yr Treasury Constant Maturity Rate,30Yr Fixed Rate Mortgage Avg in US,Case-Shiller NY Home Price Index,Case-Shiller US National Home Price Index,Consumer Price Index,Core CPI,Effective Fed Funds Rate,Inflation Rate,Population Level,Real Gross Domestic Product,SP500 Index,Unemployment Rate,US Real Median Household Income,10Yr Treasury Constant Maturity Rate %Chg,30Yr Fixed Rate Mortgage Avg in US %Chg,Case-Shiller NY Home Price Index %Chg,Case-Shiller US National Home Price Index %Chg,Consumer Price Index %Chg,Core CPI %Chg,Effective Fed Funds Rate %Chg,Inflation Rate %Chg,Population Level %Chg,Real Gross Domestic Product %Chg,SP500 Index %Chg,Unemployment Rate %Chg,US Real Median Household Income %Chg
0,2,BATHGATE,01 ONE FAMILY DWELLINGS,03028,0025,412 EAST 179TH STREET,,10457.0,1.0,0.0,1.0,1842.0,2048.0,1901.0,1,A5,355000,2013-07-08,2030280025,03028,0025,2013/14,2030280025,2,03028,0025,SANDRA FERNANDEZ,A5,1,18,100,E,3.0,258000,7496,8557,412 EAST 179 STREET,10457.0,18,32,NaN,NaN,40.850339,-73.898762,206.0,15.0,379.0,2011104.0,Mount Hope,POINT (-73.898762 40.850339),7,2013,2.579565,4.3700,167.898883,155.630,232.900,233.880,0.092581,1.543528,316405.0,16531.685000,1666.359565,7.3,58452.5,0.121550,0.073710,0.004885,0.009156,0.001957,0.002271,-0.011595,0.00857,0.000642,0.002598,0.029397,-0.026667,-0.001286
1,2,BATHGATE,01 ONE FAMILY DWELLINGS,03046,0052,2047 BATHGATE AVENUE,,10457.0,1.0,0.0,1.0,2329.0,1431.0,1901.0,1,A1,343116,2013-07-01,2030460052,03046,0052,2013/14,2030460052,2,03046,0052,MICHAEL T DONALD,A1,1,25,93,E,2.0,200000,6979,8308,2047 BATHGATE AVENUE,10457.0,20,30,NaN,NaN,40.850031,-73.895394,206.0,15.0,395.0,2011492.0,Claremont-Bathgate,POINT (-73.895394 40.850031),7,2013,2.579565,4.3700,167.898883,155.630,232.900,233.880,0.092581,1.543528,316405.0,16531.685000,1666.359565,7.3,58452.5,0.121550,0.073710,0.004885,0.009156,0.001957,0.002271,-0.011595,0.00857,0.000642,0.002598,0.029397,-0.026667,-0.001286
2,2,BAYCHESTER,01 ONE FAMILY DWELLINGS,04724,0006,3487 KINGSLAND AVENUE,,10469.0,1.0,0.0,1.0,2288.0,1480.0,1925.0,1,A2,235000,2013-07-30,2047240006,04724,0006,2013/14,2047240006,2,04724,0006,LIVING TRUST OF DOLOR,A2,1,25,91,NaN,1.0,338000,12317,18753,3487 KINGSLAND AVENUE,10469.0,21,42,NaN,NaN,40.879025,-73.845478,212.0,12.0,386.0,2060252.0,Eastchester-Edenwald-Baychester,POINT (-73.845478 40.879025),7,2013,2.579565,4.3700,167.898883,155.630,232.900,233.880,0.092581,1.543528,316405.0,16531.685000,1666.359565,7.3,58452.5,0.121550,0.073710,0.004885,0.009156,0.001957,0.002271,-0.011595,0.00857,0.000642,0.002598,0.029397,-0.026667,-0.001286
3,2,BAYCHESTER,01 ONE FAMILY DWELLINGS,04750,0048,3318 TIEMANN AVENUE,,10469.0,1.0,0.0,1.0,1710.0,1539.0,1950.0,1,A5,165000,2013-07-29,2047500048,04750,0048,2013/14,2047500048,2,04750,0048,"UMANAH, AHAZIAH",A5,1,18,95,G,2.0,283000,12060,16980,3318 TIEMANN AVENUE,10469.0,18,33,NaN,NaN,40.876008,-73.842809,212.0,12.0,358.0,2061379.0,Eastchester-Edenwald-Baychester,POINT (-73.842809 40.876008),7,2013,2.579565,4.3700,167.898883,155.630,232.900,233.880,0.092581,1.543528,316405.0,16531.685000,1666.359565,7.3,58452.5,0.121550,0.073710,0.004885,0.009156,0.001957,0.002271,-0.011595,0.00857,0.000642,0.002598,0.029397,-0.026667,-0.001286
4,2,BAYCHESTER,01 ONE FAMILY DWELLINGS,04792,0012,2837 GUNTHER AVENUE,,10469.0,1.0,0.0,1.0,4750.0,1063.0,1910.0,1,A1,205000,2013-07-30,2047920012,04792,0012,2013/14,20479

In [14]:
sales_per_month = df.groupby(['S_sale_year', 'S_sale_month'])['S_sale_date'].count()

# temporarily set pandas options to expand all rows
pd.options.display.max_rows = None
print(sales_per_month)

# reset the pandas option
pd.reset_option('display.max_rows')

S_sale_year  S_sale_month
2013         1               3499
             2               3286
             3               3580
             4               3935
             5               4627
             6               4781
             7               5169
             8               5199
             9               4502
             10              5130
             11              4202
             12              4634
2014         1               4125
             2               3573
             3               4017
             4               3876
             5               4349
             6               4639
             7               5195
             8               4615
             9               4665
             10              4728
             11              3853
             12              4637
2015         1               3744
             2               3759
             3               4212
             4               4038
             5        

## Write sales-valuation-macroeconomic merge result to pickle file

In [15]:
# make a copy of the merging progress so far
sales_val_macro = merged

# write as pickle file
pd.to_pickle(sales_val_macro, path + 'merged-sales-val-macro.pkl')

In [17]:
# also write as CSV
sales_val_macro.to_csv(path + 'merged-sales-val-macro.csv')

## Read in the NYC population data

In [18]:
# import NYC population datasets
nyc_demographic = pd.read_csv(path + 'nyc_population_data_social_by_year_boro.csv')

# TODO: import other population datasets and concatenate together

In [19]:
# add prefix to all cols in the NYC demographic dataset
nyc_demographic.rename(columns=lambda x: 'd_' + x, inplace=True)

In [20]:
display(nyc_demographic)

,d_year,d_region,d_pop_25_years_and_over,d_pop_25_years_and_over_pct,d_less_than_9th_grade,d_less_than_9th_grade_pct,d_9th_to_12th_grade_no_diploma,d_9th_to_12th_grade_no_diploma_pct,d_high_school_graduate,d_high_school_graduate_pct,d_some_college_no_degree,d_some_college_no_degree_pct,d_associates_degree,d_associates_degree_pct,d_bachelors_degree,d_bachelors_degree_pct,d_graduate_or_professional_degree,d_graduate_or_professional_degree_pct
0,2013,BK,1733746,1733746,187244.568,0.108,180309.584,0.104,450773.960,0.260,239256.948,0.138,105758.506,0.061,343281.708,0.198,225386.980,0.130
1,2013,BX,887739,887739,127834.416,0.144,126946.677,0.143,248566.920,0.280,161568.498,0.182,58590.774,0.066,102977.724,0.116,61253.991,0.069
2,2013,MN,1230210,1230210,91035.540,0.074,72582.390,0.059,161157.510,0.131,120560.580,0.098,40596.930,0.033,405969.300,0.330,337077.540,0.274
3,2013,QN,1615683,1615683,177725.130,0.110,134101.689,0.083,442697.142,0.274,231042.669,0.143,140564.421,0.087,311826.819,0.193,179340.813,0.111
4,2013,SI,321308,321308,14780.168,0.046,20563.712,0.064,104103.792,0.324,56550.208,0.176,26668.564,0.083,58478.056,0.182,40484.808,0.126
5,2014,BK,1763658,1763658,174602.142,0.099,178129.458,0.101,456787.422,0.259,239857.488,0.136,109346.796,0.062,373895.496,0.212,231039.198,0.131
6,2014,BX,907037,907037,142404.809,0.157,125171.106,0.138,247621.101,0.273,153289.253,0.169,63492.590,0.070,113379.625,0.125,60771.479,0.067
7,2014,MN,1238675,1238675,86707.250,0.070,76797.850,0.062,159789.075,0.129,121390.150,0.098,52024.350,0.042,388943.950,0.314,353022.375,0.285
8,2014,QN,1640590,1640590,172261.950,0.105,149293.690,0.091,452802.840,0.276,246088.500,0.150,132887.790,0.081,308430.920,0.188,180464.900,0.110
9,2014,SI,324356,324356,15569.088,0.048,22380.564,0.069,104118.276,0.321,53843.096,0.166,27245.904,0.084,62276.352,0.192,38598.364,0.119


## Process sales data: map borough IDs to borough codes to prepare for merge with NYC demographic data

In [21]:
# map boro codes in the sales data into two-letter codes
'''
1 = Manhattan
2 = Bronx
3 = Brooklyn
4 = Queens
5 = Staten Island
'''

def map_boro_id_to_code(boro_id):
    if boro_id == 1:
        return "MN"
    elif boro_id == 2:
        return "BX"
    elif boro_id == 3:
        return "BK"
    elif boro_id == 4:
        return "QN"
    elif boro_id == 5:
        return "SI"

merged['S_borough_code'] = merged['S_borough'].map(lambda boro_id : map_boro_id_to_code(boro_id))

# validate mapping
display(merged[['S_borough', 'S_borough_code']])

,S_borough,S_borough_code
0,2,BX
1,2,BX
2,2,BX
3,2,BX
4,2,BX
...,...,...
340535,5,SI
340536,5,SI
340537,5,SI
340538,5,SI


## Join sales-valuation-macroeconomic data with NYC demographic data on (YEAR, BORO)

In [22]:
# join sales-valuation-macro df with nyc demographic data on (year, boro)
merged = pd.merge(left=merged,
                  right=nyc_demographic,
                  how='inner',
                  left_on=['S_sale_year', 'S_borough_code'],
                  right_on=['d_year', 'd_region'])

In [23]:
# validate join
display(merged[['S_sale_year', 'S_borough_code', 'd_year', 'd_region']])

,S_sale_year,S_borough_code,d_year,d_region
0,2013,BX,2013,BX
1,2013,BX,2013,BX
2,2013,BX,2013,BX
3,2013,BX,2013,BX
4,2013,BX,2013,BX
...,...,...,...,...
316591,2018,SI,2018,SI
316592,2018,SI,2018,SI
316593,2018,SI,2018,SI
316594,2018,SI,2018,SI


In [24]:
# drop the redundant d_year and d_region cols in the merged df
merged = merged.drop(['d_year', 'd_region'], 1)

## Write sales-valuation-macro-demographic data into pickle file

In [25]:
# check the overall df
display(merged)

,S_borough,S_neighborhood,S_building_class_category,S_block,S_lot,S_address,S_apt_number,S_zipcode,S_residential_units,S_commercial_units,S_total_units,S_land_sqft,S_gross_sqft,S_year_built,S_tax_class_at_sale,S_building_class_at_sale,S_sale_price,S_sale_date,bbl,bbl_block,bbl_lot,year,v_bble,v_boro,v_block,v_lot,v_owner,v_bldgcl,v_taxclass,v_ltfront,v_ltdepth,v_ext,v_stories,v_fullval,v_avland,v_avtot,v_staddr,v_postcode,v_bldfront,v_blddepth,v_avland2,v_avtot2,v_latitude,v_longitude,v_community_board,v_council_district,v_census_tract,v_bin,v_nta,v_new_georeferenced_column,S_sale_month,S_sale_year,10Yr Treasury Constant Maturity Rate,30Yr Fixed Rate Mortgage Avg in US,Case-Shiller NY Home Price Index,Case-Shiller US National Home Price Index,Consumer Price Index,Core CPI,Effective Fed Funds Rate,Inflation Rate,Population Level,Real Gross Domestic Product,SP500 Index,Unemployment Rate,US Real Median Household Income,10Yr Treasury Constant Maturity Rate %Chg,30Yr Fixed Rate Mortgage Avg in US %Chg,Case-Shiller NY Home Price Index %Chg,Case-Shiller US National Home Price Index %Chg,Consumer Price Index %Chg,Core CPI %Chg,Effective Fed Funds Rate %Chg,Inflation Rate %Chg,Population Level %Chg,Real Gross Domestic Product %Chg,SP500 Index %Chg,Unemployment Rate %Chg,US Real Median Household Income %Chg,S_borough_code,d_pop_25_years_and_over,d_pop_25_years_and_over_pct,d_less_than_9th_grade,d_less_than_9th_grade_pct,d_9th_to_12th_grade_no_diploma,d_9th_to_12th_grade_no_diploma_pct,d_high_school_graduate,d_high_school_graduate_pct,d_some_college_no_degree,d_some_college_no_degree_pct,d_associates_degree,d_associates_degree_pct,d_bachelors_degree,d_bachelors_degree_pct,d_graduate_or_professional_degree,d_graduate_or_professional_degree_pct
0,2,BATHGATE,01 ONE FAMILY DWELLINGS,03028,0025,412 EAST 179TH STREET,,10457.0,1.0,0.0,1.0,1842.0,2048.0,1901.0,1,A5,355000,2013-07-08,2030280025,03028,0025,2013/14,2030280025,2,03028,0025,SANDRA FERNANDEZ,A5,1,18,100,E,3.0,258000,7496,8557,412 EAST 179 STREET,10457.0,18,32,NaN,NaN,40.850339,-73.898762,206.0,15.0,379.0,2011104.0,Mount Hope,POINT (-73.898762 40.850339),7,2013,2.579565,4.37,167.898883,155.630,232.900,233.880,0.092581,1.543528,316405.0,16531.685,1666.359565,7.3,58452.50,0.121550,0.07371,0.004885,0.009156,0.001957,0.002271,-0.011595,0.008570,0.000642,0.002598,0.029397,-0.026667,-0.001286,BX,887739,887739,127834.416,0.144,126946.677,0.143,248566.920,0.280,161568.498,0.182,58590.774,0.066,102977.724,0.116,61253.991,0.069
1,2,BATHGATE,01 ONE FAMILY DWELLINGS,03046,0052,2047 BATHGATE AVENUE,,10457.0,1.0,0.0,1.0,2329.0,1431.0,1901.0,1,A1,343116,2013-07-01,2030460052,03046,0052,2013/14,2030460052,2,03046,0052,MICHAEL T DONALD,A1,1,25,93,E,2.0,200000,6979,8308,2047 BATHGATE AVENUE,10457.0,20,30,NaN,NaN,40.850031,-73.895394,206.0,15.0,395.0,2011492.0,Claremont-Bathgate,POINT (-73.895394 40.850031),7,2013,2.579565,4.37,167.898883,155.630,232.900,233.880,0.092581,1.543528,316405.0,16531.685,1666.359565,7.3,58452.50,0.121550,0.07371,0.004885,0.009156,0.001957,0.002271,-0.011595,0.008570,0.000642,0.002598,0.029397,-0.026667,-0.001286,BX,887739,887739,127834.416,0.144,126946.677,0.143,248566.920,0.280,161568.498,0.182,58590.774,0.066,102977.724,0.116,61253.991,0.069
2,2,BAYCHESTER,01 ONE FAMILY DWELLINGS,04724,0006,3487 KINGSLAND AVENUE,,10469.0,1.0,0.0,1.0,2288.0,1480.0,1925.0,1,A2,235000,2013-07-30,2047240006,04724,0006,2013/14,2047240006,2,04724,0006,LIVING TRUST OF DOLOR,A2,1,25,91,NaN,1.0,338000,12317,18753,3487 KINGSLAND AVENUE,10469.0,21,42,NaN,NaN,40.879025,-73.845478,212.0,12.0,386.0,2060252.0,Eastchester-Edenwald-Baychester,POINT (-73.845478 40.879025),7,2013,2.579565,4.37,167.898883,155.630,232.900,233.880,0.092581,1.543528,316405.0,16531.685,1666.359565,7.3,58452.50,0.121550,0.07371,0.004885,0.009156,0.001957,0.002271,-0.011595,0.008570,0.000642,0.002598,0.029397,-0.026667,-0.001286,BX,887739,887739,127834.416,0.144,126946.677,0.143,248566.920,0.280,161568.498,0.182,58590.774,0.066,10297

In [27]:
# make a copy of the merging progress so far
sales_val_macro_dem = merged

# write as pickle file
pd.to_pickle(sales_val_macro_dem, path + 'merged-sales-val-macro-dem.pkl')

In [28]:
# also write as a CSV file
sales_val_macro_dem.to_csv(path + 'merged-sales-val-macro-dem.csv')